# Dendritic Spine Clustering

In [ ]:
from spine_analysis.spine.grouping import SpineGrouping
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from spine_clusterization import SpineClusterizer, DBSCANSpineClusterizer
from notebook_widgets import SpineMeshDataset, intersection_ratios_mean_distance, create_dir
from spine_segmentation import apply_scale
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score
from typing import Optional
from scipy.spatial.distance import jensenshannon
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)


dataset_path = "0.025 0.025 0.1 dataset"
scale = (1, 1, 1)
show_reduction_method = "tsne"
    
# load meshes and apply scale
spine_dataset = SpineMeshDataset().load(dataset_path)
spine_dataset.apply_scale(scale)

# load merged and reduced manual classification
manual_classification = SpineGrouping().load(f"{dataset_path}/manual_classification/manual_classification_merged_reduced.json")


In [ ]:
# load metrics
import csv
import ctypes as ct

import spine_analysis

import importlib

importlib.reload(spine_analysis)
importlib.reload(spine_analysis.shape_metric.approximation_metric)
from spine_analysis.shape_metric.approximation_metric import LightFieldZernikeMomentsSpineMetric

csv.field_size_limit(int(ct.c_ulong(-1).value // 2))
metrics_path = "output/clustering_normalized_d=25/metrics.csv"
spine_metrics_lf = SpineMetricDataset().load(metrics_path)#f"{dataset_path}/metrics.csv")
lf_name = 'LightFieldZernikeMoments'
meshes_names = list(spine_dataset.spine_names)

In [ ]:
import numpy as np
distanses = np.identity((len(meshes_names)))
distanses *= -1
for i, name_1 in enumerate(meshes_names):
  for j, name_2 in enumerate(meshes_names):
    name_1 = name_1.replace('0.025 0.025 0.1 dataset', '0.025-0.025-0.1-dataset')
    name_2 = name_2.replace('0.025 0.025 0.1 dataset', '0.025-0.025-0.1-dataset')
    if i == j:
      continue
    distanses[i,j] = LightFieldZernikeMomentsSpineMetric.distance(spine_metrics_lf.element(name_1, lf_name), spine_metrics_lf.element(name_2, lf_name))
print(distanses[distanses >= 0].min(), distanses.max())
distanses[distanses <= 0] = 10e15

k = meshes_names.index('0.025 0.025 0.1 dataset/1011-1/spine_1.off')
l = meshes_names.index('0.025 0.025 0.1 dataset/1003-22/spine_1.off')
print(k, l)
print(distanses[k,l], distanses[l,k])

index = distanses.argmin()
i = index // len(meshes_names)
j = index % len(meshes_names)
print(distanses[i,j], distanses[j,i])
print(meshes_names[i], meshes_names[j])

In [ ]:
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from notebook_widgets import SpineMeshDataset, intersection_ratios_mean_distance, create_dir
from spine_segmentation import apply_scale
from spine_analysis.spine.grouping import SpineGrouping
from spine_analysis.clusterization import SpineClusterizer, DBSCANSpineClusterizer
import numpy as np
from sklearn.neighbors import NearestNeighbors


dataset_path = "0.025 0.025 0.1 dataset"
scale = (1, 1, 1)
    
# load meshes and apply scale
spine_dataset = SpineMeshDataset().load(dataset_path)
spine_dataset.apply_scale(scale)

# load merged and reduced manual classification
manual_classification = SpineGrouping().load(f"{dataset_path}/manual_classification/manual_classification_merged_reduced.json")
manual_classification = manual_classification.get_spines_subset(spine_dataset.spine_names)

# load metrics
spine_metrics = SpineMetricDataset().load(f"{dataset_path}/metrics.csv")
spine_metrics = spine_metrics.get_spines_subset(manual_classification.samples)

# extract metric subsets OldChordDistribution,OpenAngle,CVD,AverageDistance,LengthVolumeRatio,LengthAreaRatio,JunctionArea,Length,Area,Volume,ConvexHullVolume,ConvexHullRatio
classic = spine_metrics.get_metrics_subset(['OpenAngle', 'CVD', 'AverageDistance', 'Length', 'Area', 'Volume', 'ConvexHullVolume', 'ConvexHullRatio'])
# extract metric subsets
classic = spine_metrics.get_metrics_subset(['OpenAngle', 'CVD', "JunctionArea", 'AverageDistance', 'Length', 'Area', 'Volume', 'ConvexHullVolume', 'ConvexHullRatio', "LengthVolumeRatio", "LengthAreaRatio"])
chord = spine_metrics.get_metrics_subset(['OldChordDistribution'])

# set score function to mean distance between class over cluster distributions
#score_func = lambda clusterizer: intersection_ratios_mean_distance(manual_classification, clusterizer.grouping, False)


# prepare folders for export
create_dir(f"{dataset_path}/clustering")
classic_save_path = f"{dataset_path}/clustering/classic"
create_dir(classic_save_path)
chord_save_path = f"{dataset_path}/clustering/chord/euclidean"
create_dir(f"{dataset_path}/clustering/chord")
create_dir(f"{dataset_path}/clustering/chord/euclidean")
chord_js_save_path = f"{dataset_path}/clustering/chord/jensen-shannon"
create_dir(f"{dataset_path}/clustering/chord/jensen-shannon")

# elbow method
def kmeans_elbow_score(clusterizer: SpineClusterizer) -> float:
    # sum of mean distances to cluster center
    output = 0
    for group in clusterizer.grouping.groups.values():
        center = sum(clusterizer.fit_metrics.row_as_array(spine_name) for spine_name in group)
        output += sum(np.inner(center - clusterizer.fit_metrics.row_as_array(spine_name),
                               center - clusterizer.fit_metrics.row_as_array(spine_name)) for spine_name in group)
    return output


def dbscan_elbow_score(clusterizer: DBSCANSpineClusterizer) -> float:
    # number of points with not enough neighbours close enough to form a cluster
    neigh = NearestNeighbors(n_neighbors=clusterizer.min_samples, metric=clusterizer.metric)
    data = clusterizer.fit_metrics.as_array()
    nbrs = neigh.fit(data)
    distances, indices = nbrs.kneighbors(data)
    # get distances to closest k-th neighbour
    distances = distances[:, -1]
    # sort distances in descending order
    distances = -np.sort(-distances, axis=0)
    for i in range(len(distances)):
        if clusterizer.eps > distances[i]:
            return i
    return len(distances)

def silhouette(clusterizer: SpineClusterizer, metric: Optional[callable] = None) -> float:
    datas = []
    labels = []
    for i, group in enumerate(clusterizer.grouping.groups.values()):
        datas.extend(clusterizer.fit_metrics.row_as_array(spine) for spine in group)
        labels.extend([i for _ in group])
    
    labels = np.array(labels)
    if metric is None:
        score = silhouette_score(datas, labels, metric=clusterizer.metric)
    else:
        score = silhouette_score(np.array([[metric(x1, x2) for x1 in datas] for x2 in datas]), labels, metric="precomputed")
    return score

def js_distance(x, y) -> float:
    return np.sqrt(jensenshannon(x, y))

## k-Means Classic Metrics

In [ ]:
from notebook_widgets import k_means_clustering_experiment_widget

# score_func = lambda clusterizer: intersection_ratios_mean_distance(manual_classification, clusterizer.grouping, False)
#score_func = silhouette
score_func = kmeans_elbow_score

dim_reduction = ""

display(k_means_clustering_experiment_widget(classic, spine_metrics, spine_dataset, score_func,
                                             max_num_of_clusters=100, classification=manual_classification,
                                             save_folder=classic_save_path, dim_reduction=dim_reduction, show_method=show_reduction_method))

## k-Means Chord Histograms

In [ ]:
from notebook_widgets import k_means_clustering_experiment_widget

# score_func = lambda clusterizer: intersection_ratios_mean_distance(manual_classification, clusterizer.grouping, False)
score_func = kmeans_elbow_score

display(k_means_clustering_experiment_widget(chord, spine_metrics, spine_dataset, score_func,
                                             max_num_of_clusters=100, classification=manual_classification,
                                             save_folder=chord_save_path, dim_reduction="", show_method=show_reduction_method))

## DBSCAN Classic Metrics

In [ ]:
from notebook_widgets import dbscan_clustering_experiment_widget

min_eps = 0.2
max_eps = 6
eps_step = 0.1

# score_func = lambda clusterizer: intersection_ratios_mean_distance(manual_classification, clusterizer.grouping, False)
score_func = dbscan_elbow_score

display(dbscan_clustering_experiment_widget(classic, spine_metrics, spine_dataset, score_func,
                                            min_eps=min_eps, max_eps=max_eps, eps_step=eps_step, dim_reduction="pca", show_method=show_reduction_method,
                                            classification=manual_classification, save_folder=classic_save_path))

## DBSCAN Chord Histograms Euclidean Distance

In [ ]:
from notebook_widgets import dbscan_clustering_experiment_widget

min_eps = 0.1
max_eps = 10
eps_step = 0.1

# score_func = lambda clusterizer: intersection_ratios_mean_distance(manual_classification, clusterizer.grouping, False)
score_func = dbscan_elbow_score

display(dbscan_clustering_experiment_widget(chord, spine_metrics, spine_dataset, score_func,
                                            min_eps=min_eps, max_eps=max_eps, eps_step=eps_step, dim_reduction="", show_method=show_reduction_method,
                                            classification=manual_classification, save_folder=chord_save_path))

## DBSCAN Chord Histograms Jensen — Shannon Distance

In [ ]:
from notebook_widgets import dbscan_clustering_experiment_widget
from scipy.spatial.distance import jensenshannon
import numpy as np

min_eps = 0.1
max_eps = 1
eps_step = 0.01
use_pca = False

# score_func = lambda clusterizer: intersection_ratios_mean_distance(manual_classification, clusterizer.grouping, False)
score_func = dbscan_elbow_score

def js_distance(x, y) -> float:
    return np.sqrt(jensenshannon(x, y))

display(dbscan_clustering_experiment_widget(chord, spine_metrics, spine_dataset, score_func, metric=js_distance,
                                            min_eps=min_eps, max_eps=max_eps, eps_step=eps_step, use_pca=use_pca,
                                            classification=manual_classification, save_folder=chord_js_save_path))

## K-Means Spherical Harmonics descriptor

In [ ]:
dataset_folder = '0.025-0.025-0.1-dataset'
output_dir = "output/clustering_normalized_shpHarm"
cur_metrics = ["SphericalGarmonics"]
shpHarm_prefix = f"sphHarm"
calculate_metrics = False
save_metrics = True
standardize_metrics = True
scale = (1, 1, 5)

In [ ]:
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from notebook_widgets import k_means_clustering_experiment_widget, create_dir, SpineMeshDataset

create_dir(output_dir)
spine_dataset = SpineMeshDataset().load(dataset_folder)
spine_dataset.apply_scale(scale)

if calculate_metrics:
    every_spine_metrics = SpineMetricDataset()
    every_spine_metrics.calculate_metrics(spine_dataset.spine_meshes, cur_metrics)

    if save_metrics:
        every_spine_metrics.save(f"{output_dir}/metrics.csv")

every_spine_metrics = SpineMetricDataset.load(f"{output_dir}/metrics.csv")
if standardize_metrics:
    every_spine_metrics.standardize()

sphHarm = every_spine_metrics.get_metrics_subset(cur_metrics)
score_func = lambda clusterizer: 1

In [ ]:
from spine_analysis.shape_metric import LightFieldZernikeMomentsSpineMetric
from notebook_widgets import kernel_k_means_clustering_experiment_widget
from notebook_widgets import SpineMeshDataset
from scipy.spatial import distance

# load meshes and apply scale
spine_dataset = SpineMeshDataset().load(dataset_folder)
display(k_means_clustering_experiment_widget(sphHarm, every_spine_metrics, spine_dataset, score_func, metric=distance.cityblock,
                                             max_num_of_clusters=5, use_pca=False,
                                             filename_prefix="sphHarm"))

## Kernel k-Means Zernike Moments

In [ ]:
dataset_folder = '0.025-0.025-0.1-dataset'
output_dir = "output/clustering_normalized"
cur_metrics = ["LightFieldZernikeMoments"]
calculate_metrics = False
save_metrics = True
standardize_metrics = True

In [ ]:
from spine_analysis.mesh.utils import load_spine_meshes
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from notebook_widgets import k_means_clustering_experiment_widget, create_dir

create_dir(output_dir)
spine_meshes = load_spine_meshes(folder_path=dataset_folder)

if calculate_metrics:
    every_spine_metrics = SpineMetricDataset()
    every_spine_metrics.calculate_metrics(spine_meshes, cur_metrics)

    if save_metrics:
        every_spine_metrics.save(f"{output_dir}/metrics.csv")

In [ ]:
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from spine_analysis.clusterization.postprocess import score as score_func
from sklearn.metrics import silhouette_score

every_spine_metrics = SpineMetricDataset.load(f"{output_dir}/metrics.csv")
if standardize_metrics:
    every_spine_metrics.standardize()

zernike = every_spine_metrics.get_metrics_subset(['LightFieldZernikeMoments'])
zernike_real = every_spine_metrics.clasterization_preprocess(zernike_postprocess='real')

In [ ]:
from spine_analysis.shape_metric import LightFieldZernikeMomentsSpineMetric
from notebook_widgets import kernel_k_means_clustering_experiment_widget
from notebook_widgets import SpineMeshDataset

# load meshes and apply scale
spine_dataset = SpineMeshDataset().load(dataset_folder)
display(kernel_k_means_clustering_experiment_widget(zernike, every_spine_metrics, spine_dataset, score_func, metric=LightFieldZernikeMomentsSpineMetric.repr_distance,
                                             max_num_of_clusters=4, use_pca=False,
                                             filename_prefix="zernike"))

## View clustering

In [ ]:
from notebook_widgets import inspect_saved_groupings_widget

display(inspect_saved_groupings_widget(f"{dataset_path}/clustering", spine_dataset, spine_metrics,
                                       chord, classic, manual_classification))